"Este trabalho utilizou dados disponibilizados pelo repositório COVID-19 Data Sharing/BR, disponível em: “ https://repositoriodatasharingfapesp.uspdigital.usp.br/".



In [1]:
import numpy as np
import pandas as pd

In [2]:
data_sources =\
[
   ("dados/initial/einstein_e.csv", ','),
   ("dados/initial/fleury_e.csv", ','),
   ("dados/initial/hsl_e.csv", ','),
   ("dados/added/einstein/dataset_exames.csv", '|'),
   ("dados/added/hsl/hsl_lab_result_1.csv", '|'),
   ("dados/added/fleury/dataset_exames.csv", '|'),
]

dfs_raw = [pd.read_csv(df_file, separator, encoding="latin-1") for df_file, separator in data_sources]

In [6]:
for df in dfs_raw:
    print(list(df.columns))

['id_paciente', 'dt_coleta', 'de_origem', 'de_exame', 'de_analito', 'de_resultado', 'cd_unidade', 'de_valor_referencia']
['ID_PACIENTE', 'DT_COLETA', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'DE_RESULTADO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA']
['Ã¯Â»Â¿ID_PACIENTE', 'ID_ATENDIMENTO', 'DT_COLETA', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'DE_RESULTADO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA']
['id_paciente', 'dt_coleta', 'de_origem', 'de_exame', 'de_analito', 'de_resultado', 'cd_unidade', 'de_valor_referencia']
['ID_PACIENTE', 'ID_ATENDIMENTO', 'DT_COLETA', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'DE_RESULTADO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA']
['ID_PACIENTE', 'DT_COLETA', 'DE_ORIGEM', 'DE_EXAME', 'DE_ANALITO', 'DE_RESULTADO', 'CD_UNIDADE', 'DE_VALOR_REFERENCIA']


In [26]:
len(dfini_f_exames_raw["ID_PACIENTE"].unique())

100

In [27]:
l = [str(ex) for ex in dfini_f_exames_raw["DE_EXAME"].unique().copy()]
l = sorted(l)
for ex in l:
    print(ex)


25 HIDROXI-VITAMINA D: soro
ACIDO ASCORBICO: plasma
ACIDO FOLICO: soro
ACIDO URICO: soro
ALBUMINA: soro
ALBUMINURIA: amostra isolada
ALFA-1 GLICOPROTEINA ACIDA: soro
ALFA-FETOPROTEINA: soro
AMILASE: soro
ANDROSTENEDIONA: soro
ANTICORPO ANTI-RECEPTOR DE TSH (TRAB): soro
ANTICORPO CONTRA O ANTIGENO DO CORE DO VIRUS DA HEPATITE B: soro
ANTICORPOS ANTI-PEROXIDASE TIROIDIANA: soro
ANTICORPOS ANTI-TIROGLOBULINA: soro
ANTICORPOS ANTI-VIRUS DA HEPATITE C (Anti-HCV): soro
ANTICORPOS DA CLASSE IgA ANTI-TRANSGLUTAMINASE TECIDUAL: soro
ANTICORPOS IRREGULARES ANTI-ERITROCITARIOS: soro
ANTICORPOS TOTAIS CONTRA O VIRUS DA HEPATITE A (Anti-VHA Total)
ANTIGENO CARCINOEMBRIONICO (CEA): soro
ANTIGENO DE SUPERFICIE DA HEPATITE B: soro
ANTIGENO PROSTATICO ESPECIFICO(PSA): soro
ANTISTREPTOLISINA O (ASLO): ANTICORPOS: soro
APOLIPOPROTEINA A-1: soro
APOLIPOPROTEINA B: soro
BILIRRUBINAS: soro
CA 15-3: soro
CA 19-9: soro
CA 72-4: soro
CA-125: soro
CALCIO IONIZADO: soro
CALCIO: soro
CHLAMYDIA TRACHOMATIS: ANTICO